# GENERAL TITLE

## Short description of the notebook

# General Setup

In [10]:
%%bash
cd /content

REPO=https://github.com/jacopo-raffaeli/portfolio-replica.git
DIR=portfolio-replica

# Clone if needed, else pull latest
if [ ! -d "$DIR" ]; then
  git clone $REPO > /dev/null 2>&1
else
  cd $DIR
  git pull origin main > /dev/null 2>&1
  cd ..
fi

# Enter project root and install dependencies
cd $DIR
pip install -r requirements.txt > /dev/null 2>&1


In [37]:
# Standard library
import os
import sys
import warnings
import random
import logging
import pickle

# Core scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
# Add repo root to sys.path for imports
PROJECT_ROOT = "/content/portfolio-replica"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    sys.path.append(os.path.join(PROJECT_ROOT, 'src'))

# Set working directory for relative paths
os.chdir(PROJECT_ROOT)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/portfolio-replica


In [13]:
# Suppress warnings
warnings.filterwarnings('ignore')

# Pandas display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 3)

# Seaborn and Matplotlib display options
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Set reproducible seeds
random.seed(42)
np.random.seed(42)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S'
)

# Load & Inspect Dataset

In [38]:
# Define paths
data_raw_path = "data/raw/"
data_interim_path = "data/interim/"
data_processed_path = "data/processed/"

# Read full names (row 4)
full_names_df = pd.read_excel(data_raw_path + "dataset_raw.xlsx", header=None, skiprows=3, nrows=1)
full_names = full_names_df.iloc[0].tolist()[1:]
# Read Bloomberg tickers (row 5)
tickers_df = pd.read_excel(data_raw_path + "dataset_raw.xlsx", header=None, skiprows=4, nrows=1)
tickers = tickers_df.iloc[0].tolist()[1:]
# Create and save a dictionary mapping tickers to full names
tickers_name_dict = dict(zip(tickers, full_names))
with open(data_processed_path + "tickers_name_dict.pkl", "wb") as f:
    pickle.dump(tickers_name_dict, f)

# Read the actual data without using any row as a header (from row 6)
df_raw = pd.read_excel(data_raw_path + "dataset_raw.xlsx", header=None, skiprows=5)
# Set column names and index
df_raw.columns = ['Date'] + tickers
df_raw['Date'] = pd.to_datetime(df_raw['Date'], format='%d/%m/%Y')
df_raw = df_raw.set_index('Date')

In [62]:
print("Dataset shape:", df_raw.shape)
display(df_raw.head())

Dataset shape: (705, 15)


,MXWO,MXWD,LEGATRUU,HFRXGL,RX1,TY1,GC1,CO1,ES1,VG1,NQ1,LLL1,TP1,DU1,TU2
Date,,,,,,,,,,,,,,,
2007-10-23,1633.44,414.14,350.214,1343.63,113.70,110.516,759.0,82.85,1525.50,4393.0,2212.00,1287.0,1570.5,103.385,103.719
2007-10-30,1663.89,423.26,352.541,1356.53,113.79,110.656,787.8,87.44,1536.00,4476.0,2217.75,1323.9,1610.5,103.410,103.812
2007-11-06,1651.59,419.51,354.176,1360.20,113.79,110.875,823.4,93.26,1525.00,4425.0,2233.50,1320.0,1575.5,103.410,104.047
2007-11-13,1601.81,405.98,357.222,1347.16,114.35,111.719,799.0,88.83,1483.25,4323.0,2066.75,1271.0,1440.5,103.595,104.305
2007-11-20,1570.74,398.54,359.445,1335.21,114.72,113.156,791.4,95.49,1446.00,4296.0,2035.50,1225.2,1472.5,103.800,104.945


In [65]:
print("Dataset Statistics:")
display(df_raw.describe())

Dataset Statistics:


,MXWO,MXWD,LEGATRUU,HFRXGL,RX1,TY1,GC1,CO1,ES1,VG1,NQ1,LLL1,TP1,DU1,TU2
count,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000,705.000
mean,1644.126,404.152,452.203,1210.092,147.279,125.967,1317.903,77.018,1960.443,3072.168,4526.981,1064.444,1277.263,110.221,108.628
std,443.390,100.980,47.338,73.965,19.319,6.295,274.478,26.794,770.071,473.813,2900.867,128.955,353.598,2.375,1.533
min,705.350,176.100,342.247,1021.400,110.200,110.516,732.700,19.330,689.500,1873.000,1072.000,465.100,704.000,102.190,103.719
25%,1293.430,328.790,431.737,1158.150,128.900,121.203,1178.000,54.270,1314.500,2753.000,2244.000,1103.600,905.000,109.230,107.871
50%,1646.690,402.570,456.729,1215.200,148.000,126.000,1283.000,72.050,1909.250,3071.000,3884.750,1103.600,1302.500,110.855,109.094
75%,1978.830,478.410,478.187,1254.250,163.240,130.344,1499.700,105.780,2569.750,3433.000,6333.250,1103.600,1587.500,111.990,109.906
max,2944.050,702.990,559.312,1415.110,179.320,140.328,2001.200,140.670,4132.800,4476.000,13975.750,1323.900,1978.000,112.720,110.527


In [64]:
print("Missing Values:")
display(df_raw.isna().sum().sort_values(ascending=False))

Missing Values:


,0
MXWO,0
MXWD,0
LEGATRUU,0
HFRXGL,0
RX1,0
TY1,0
GC1,0
CO1,0
ES1,0
VG1,0


In [67]:
print(f"From {df_raw.index.min()} to {df_raw.index.max()}, total days: {df_raw.shape[0]}")
display(df_raw.index.to_series().diff().value_counts().head())

From 2007-10-23 00:00:00 to 2021-04-20 00:00:00, total days: 705


,count
Date,
7 days,704


# PUSH

In [79]:
!curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg > /dev/null
!sudo chmod go+r /usr/share/keyrings/githubcli-archive-keyring.gpg > /dev/null
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null
!sudo apt update > /dev/null
!sudo apt install gh -y > /dev/null


4+1 records in
4+1 records out
2270 bytes (2.3 kB, 2.2 KiB) copied, 0.0901815 s, 25.2 kB/s


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [80]:
!git config --global user.email "jacoporaffaeli@gmail.com"
!git config --global user.name "jacopo-raffaeli"

In [85]:
from getpass import getpass
import os

# 1. Get your GitHub Personal Access Token securely
token = getpass("Paste your GitHub Personal Access Token: ")
os.environ['GITHUB_TOKEN'] = token

# 2. Set your GitHub repo details
GITHUB_USERNAME = "jacopo-raffaeli"
REPO_NAME = "portfolio-replica"
BRANCH = "main"

# 3. Construct remote URL with token embedded (hidden from output)
remote_url = f"https://{token}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"

# 4. Set git user info (if not done already)
!git config --global user.email "jacoporaffaeli@gmail.com"
!git config --global user.name "jacopo-raffaeli"

# 5. Change remote origin URL to token-embedded one
!git remote set-url origin {remote_url}


Paste your GitHub Personal Access Token: ··········


In [86]:
# 6. Add and commit changes (customize your commit message)
!git add .
!git commit -m "feat: Add Initial EDA" || echo "No changes to commit."

# 7. Push to GitHub
!git push origin {BRANCH}

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
No changes to commit.
Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 699 bytes | 699.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/jacopo-raffaeli/portfolio-replica.git
   d13d1b1..b1d6748  main -> main


In [87]:
# Show git status, which files are changed and staged
!git status

# Show last commit files changed (to check if notebook was included)
!git show --name-only --oneline -1

# Show current branch
!git branch

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
b1d6748 (HEAD -> main, origin/main, origin/HEAD) feat: Add Initial EDA
data/processed/tickers_name_dict.pkl
* main
